# ATEK Data Proprocessing Example

### 1. Load the example adt data

In [1]:
import os

from atek.data_preprocess.sample_builders.obb3_sample_builder import ObjectDetection3dSampleBuilder
from atek.data_preprocess.wds_writer import AtekWdsWriter

from omegaconf import OmegaConf

example_adt_data_dir = "/home/louy/Calibration_data_link/Atek/2024_01_25_AseTest/adt_data_example/1WM103600M1292_optitrack_SSL_1_Aria_OS_DinoToy_seq010"
config_path = os.path.join("/home/louy/fbsource/fbcode/surreal/data_services/atek/atek/configs/obb3_preprocess_base.yaml")

conf = OmegaConf.load(config_path)

### 2. Sample builder example

In [ ]:
conf.processors.rgb.rotate_image_cw90deg = True   
conf.processors.rgb.undistort_to_linear_camera = True

conf.processors.slam_left.rotate_image_cw90deg = True   
conf.processors.slam_left.undistort_to_linear_camera = True
sample_builder = ObjectDetection3dSampleBuilder(
    conf=conf.processors,
    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
    mps_files={
        "mps_closedloop_traj_file": os.path.join(
            example_adt_data_dir, "aria_trajectory.csv"
        ),
    },
    gt_files={
        "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),
        "obb3_traj_file": os.path.join(
            example_adt_data_dir, "scene_objects.csv"
        ),
        "instance_json_file": os.path.join(
            example_adt_data_dir, "instances.json"
        ),
    },
)

# Print the frame dataclass keys
sample = sample_builder.get_sample_by_timestamp_ns(180875319411000)
from dataclasses import asdict
print(asdict(sample).keys())

### 2. Write wds files

In [ ]:
from atek.data_preprocess.wds_writer import (
    AtekWdsWriter,
)
from atek.data_preprocess.subsampling_lib.temporal_subsampler import CameraTemporalSubsampler

conf.camera_temporal_subsampler.sample_length_in_num_frames = 5
conf.camera_temporal_subsampler.stride_length_in_num_frames = 4
subsampler = CameraTemporalSubsampler(vrs_file = os.path.join(example_adt_data_dir, "video.vrs"), conf = conf.camera_temporal_subsampler)

conf.wds_writer.prefix_string = "test"
atek_wds_writer = AtekWdsWriter(output_path = "/home/louy/Calibration_data_link/Atek/2024_05_01_AtekRefactorTest/test_wds", conf = conf.wds_writer)

for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)
    print(f"sample {i} has timestamps of {timestamps_ns}")
    
    for t in timestamps_ns:
        sample = sample_builder.get_sample_by_timestamp_ns(t)
        if sample is not None:
            atek_wds_writer.add_sample(data_sample = sample)

atek_wds_writer.close()
